# H20 AutoML Kilocalorie Regression

## Abstract
This notebook is a simple example of using H20 AutoML to predict the kilocalories of common foods based on its nutritional composition. 

The model is trained using H20 AutoML and the best model is selected based on the root mean squared error (RMSE) of the test set. 

This research found that the best model to predict the kilocalories 

## Modeling with AutoML 
H2O will speed up some of the basic data processing steps, feature engineering, and model testing so I can focus on determing the best model for this dataset.

In [56]:
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [57]:
!pip3 install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [58]:
# Importing necessary libraries
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
import psutil
import random
import logging
import pandas as pd

## Virtual Machine Configuration

In [59]:
min_mem_size=6 
run_time=222

In [60]:
pct_memory=0.5
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size)

1


In [61]:
port_no=random.randint(5555,55555)
try:
  h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no) # start h2o
except:
  logging.critical('h2o.init')
  h2o.download_all_logs(dirname=logs_path, filename=logfile)    
  h2o.cluster().shutdown()
  sys.exit(2)

Checking whether there is an H2O instance running at http://localhost:14155..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.4" 2022-07-19; OpenJDK Runtime Environment Temurin-17.0.4+8 (build 17.0.4+8); OpenJDK 64-Bit Server VM Temurin-17.0.4+8 (build 17.0.4+8, mixed mode)
  Starting server from /Users/vraosharma/Desktop/Python/DSEMT/exploratory-data-analysis/venv/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/36/syxnttfx02z8nxmjh2_llhnr0000gn/T/tmp92x53rmf
  JVM stdout: /var/folders/36/syxnttfx02z8nxmjh2_llhnr0000gn/T/tmp92x53rmf/h2o_vraosharma_started_from_python.out
  JVM stderr: /var/folders/36/syxnttfx02z8nxmjh2_llhnr0000gn/T/tmp92x53rmf/h2o_vraosharma_started_from_python.err
  Server is running at http://127.0.0.1:14155
Connecting to H2O server at http://127.0.0.1:14155 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,9 days
H2O_cluster_name:,H2O_from_python_vraosharma_yf59gw
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


## Re-acquainting ourselves with the data
This project is a continuation of my last, so I'll be using the [Food Nutrition Dataset](https://www.kaggle.com/datasets/shrutisaxena/food-nutrition-dataset?resource=download&select=food.csv) (which is based on the USDA's [FoodData Central database](https://fdc.nal.usda.gov/fdc-app.html#/)) used in my Predicting Kilocalories with Linear Regression notebook (check that out first to see the profile of the data). Special thanks to SHRUTI SAXENA for compiling this dataset.

In [62]:
# Addding the dataset from github https://github.com/vraosharma-northeastern/exploratory-data-analysis/blob/main/Nutrition%20/food.csv
!wget https://raw.githubusercontent.com/vraosharma-northeastern/exploratory-data-analysis/main/Nutrition%20/food.csv

zsh:1: command not found: wget


In [65]:
url = "https://raw.githubusercontent.com/vishnuraosharma/exploratory-data-analysis/main/Nutrition%20/food.csv"
df = h2o.import_file(path = url)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [67]:
df.head()

Category,Description,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Ash,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,Data.Fiber,Data.Kilocalories,Data.Lutein and Zeaxanthin,Data.Lycopene,Data.Manganese,Data.Niacin,Data.Pantothenic Acid,Data.Protein,Data.Refuse Percentage,Data.Retinol,Data.Riboflavin,Data.Selenium,Data.Sugar Total,Data.Thiamin,Data.Water,Data.Fat.Monosaturated Fat,Data.Fat.Polysaturated Fat,Data.Fat.Saturated Fat,Data.Fat.Total Lipid,Data.Household Weights.1st Household Weight,Data.Household Weights.1st Household Weight Description,Data.Household Weights.2nd Household Weight,Data.Household Weights.2nd Household Weight Description,Data.Major Minerals.Calcium,Data.Major Minerals.Copper,Data.Major Minerals.Iron,Data.Major Minerals.Magnesium,Data.Major Minerals.Phosphorus,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - IU,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
BUTTER,"BUTTER,WITH SALT",1001,0,2.11,158,0,0.06,215,19,0,717,0,0,0,0.042,0.11,0.85,0,671,0.034,1,0.06,0.005,15.87,21.021,3.043,51.368,81.11,227,1 cup,14,1 tbsp,24,0,0.02,2,24,24,576,0.09,2499,684,0.17,0.003,0,2.32,7
BUTTER,"BUTTER,WHIPPED,WITH SALT",1002,0,2.11,158,0,0.06,219,19,0,717,0,0,0.004,0.042,0.11,0.85,0,671,0.034,1,0.06,0.005,15.87,23.426,3.012,50.489,81.11,151,1 cup,9,1 tbsp,24,0.016,0.16,2,23,26,827,0.05,2499,684,0.13,0.003,0,2.32,7
BUTTER OIL,"BUTTER OIL,ANHYDROUS",1003,0,0,193,0,0,256,22,0,876,0,0,0,0.003,0.01,0.28,0,824,0.005,0,0,0.001,0.24,28.732,3.694,61.924,99.48,205,1 cup,13,1 tbsp,4,0.001,0,0,3,5,2,0.01,3069,840,0.01,0.001,0,2.8,8.6
CHEESE,"CHEESE,BLUE",1004,0,5.11,74,0,2.34,75,15,0,353,0,0,0.009,1.016,1.729,21.4,0,192,0.382,14.5,0.5,0.029,42.41,7.778,0.8,18.669,28.74,28.35,1 oz,17,1 cubic inch,528,0.04,0.31,23,387,256,1395,2.66,763,198,1.22,0.166,0,0.25,2.4
CHEESE,"CHEESE,BRICK",1005,0,3.18,76,0,2.79,94,15,0,371,0,0,0.012,0.118,0.288,23.24,0,286,0.351,14.5,0.51,0.014,41.11,8.598,0.784,18.764,29.68,132,"1 cup, diced",113,"1 cup, shredded",674,0.024,0.43,24,451,136,560,2.6,1080,292,1.26,0.065,0,0.26,2.5
CHEESE,"CHEESE,BRIE",1006,0,2.7,9,0,0.45,100,15,0,334,0,0,0.034,0.38,0.69,20.75,0,173,0.52,14.5,0.45,0.07,48.42,8.013,0.826,17.41,27.68,240,"1 cup, melted",144,"1 cup, sliced",184,0.019,0.5,20,188,152,629,2.38,592,174,1.65,0.235,0,0.24,2.3
CHEESE,"CHEESE,CAMEMBERT",1007,0,3.68,12,0,0.46,72,15,0,300,0,0,0.038,0.63,1.364,19.8,0,240,0.488,14.5,0.46,0.028,51.8,7.023,0.724,15.259,24.26,246,1 cup,28,1 oz,388,0.021,0.33,20,347,187,842,2.38,820,241,1.3,0.227,0,0.21,2
CHEESE,"CHEESE,CARAWAY",1008,0,3.28,0,0,3.06,93,0,0,376,0,0,0.021,0.18,0.19,25.18,0,262,0.45,14.5,0,0.031,39.28,8.275,0.83,18.584,29.2,28.35,1 oz,0,,673,0.024,0.64,22,490,93,690,2.94,1054,271,0.27,0.074,0,0,0
CHEESE,"CHEESE,CHEDDAR",1009,0,3.93,85,0,1.28,105,16,0,403,0,0,0.01,0.08,0.413,24.9,0,258,0.375,13.9,0.52,0.027,36.75,9.391,0.942,21.092,33.14,132,"1 cup, diced",244,"1 cup, melted",721,0.031,0.68,28,512,98,621,3.11,1002,265,0.83,0.074,0,0.29,2.8
CHEESE,"CHEESE,CHESHIRE",1010,0,3.6,0,0,4.78,103,0,0,387,0,0,0.012,0.08,0.413,23.37,0,220,0.293,14.5,0,0.046,37.65,8.671,0.87,19.475,30.6,28.35,1 oz,0,,643,0.042,0.21,21,464,95,700,2.79,985,233,0.83,0.074,0,0,0


In [ ]:
df.describe()

## Data Preprocessing
